# Naver 영화 네티즌 평점 수집 - 미드웨이, 2019

In [1]:
from bs4 import BeautifulSoup

In [2]:
from urllib.request import urlopen

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=186615'
html = urlopen(url_base+url_sub)
soup = BeautifulSoup(html,'html.parser')

## iframe의 src 주소값을 얻어야 함

In [6]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=186615&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

### Review 건수 구하기

In [7]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')

In [8]:
count = soup.find('strong','total')
count

<strong class="total">관람객 평점 <em>4,813</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [9]:
count = count.find('em').string  #get_text() 대신에 string
count

'4,813'

In [10]:
count = int(count.replace(',',''))
count

4813

In [11]:
import math
loop_count = math.ceil(count/10)
loop_count

482

## 첫 번째 페이지에서 리뷰, 평점, 날짜 구해보기

### - 한 개의 리뷰에서 평점, 리뷰, 날짜 구하기

In [12]:
result = soup.find(class_= 'score_result')

In [30]:
lis = result.find_all('li')
li = lis[0]

In [31]:
li

<li>
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:90.0%"></span></span><em>9</em>
</div>
<div class="score_reple">
<p>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_0">
														
															
															
																미드웨이가 재밌으면 추천 백두산이 재밌으면 비추 
															
														
														
													</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(16564107, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>낄낄쿨레(sunj****)</span>
</a>
</em>
<em>2019.12.31 09:48</em>
</dt>
<dd>
<a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','sunj****', '7TAQe86ieuRBinWe2jmrHhDegNwu7MCazDF00t4hcpQ=', '미드웨이가 재밌으면 추천 백두산이 재밌으면 비추 ', '16564107', 'point_after', false);return false;"><em>신고</em></a>
</dd>
</dl>
</div>
<div class="btn_area">
<a class="_sympathyButton" href="#" onclick="parent.clickcr(this, 'ara.sym', 

In [32]:
score = int(li.find(class_='star_score').find('em').get_text())
score

9

In [33]:
review = li.find(class_= 'score_reple').find('p').find('span').get_text().strip(' \t\r\n') # ' ', \r,\t, \n 제거하기, 특정 문자 제거 방법
review

'미드웨이가 재밌으면 추천 백두산이 재밌으면 비추'

In [17]:
date = li.find(class_= 'score_reple').find_all('em')
date[1].get_text()

'2019.12.31 09:48'

### - 10개의 데이터에서 데이터 가져오기

In [35]:
point = []
review = []
date = []

for li in lis :
    score = li.find(class_='star_score').find('em').get_text()
    score = int(score)
    point.append(score)
    review.append(li.find(class_= 'score_reple').find('p').find('span').get_text().strip(' \t\r\n'))
    date.append(li.find(class_= 'score_reple').find_all('em')[1].get_text())
    
print(point)
print(review)
print(date)

[9, 10, 10, 10, 10, 10, 10, 10, 10, 8]
['미드웨이가 재밌으면 추천 백두산이 재밌으면 비추', '저 해전이 있었기에 우리나라 광복도 가능 했음', '백두산 상영관 대폭줄이고 미드웨이 상영관 대폭늘려라', '방금 개봉했는데 1점 준애는 뭐냐 ㅋㅋ 일본놈이냐? 이제 광고돌고 있을 시간이다.', '관람객', '스케일 겁나커서 보는내내 입벌리고 봣네욬ㅋㅋ재밋습니다', '안보면후회함 백두산보다낫다', '재밌으면 추천 재미없으면 비추천 눌러주십쇼 형님들', '일본의 최대 실수는 미국을 건드린거다 그 대가로 원폭 처맞고 멸망함ㅋㅋㅋ지금 하는 짓거리보면 아직 더 맞아야 될듯', '아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로...']
['2019.12.31 09:48', '2019.12.31 10:41', '2019.12.31 09:38', '2019.12.31 09:14', '2019.12.31 19:58', '2019.12.31 09:29', '2019.12.31 09:52', '2019.12.31 09:01', '2019.12.31 16:37', '2019.12.31 10:14']


In [19]:
df = pd.DataFrame({'평점':point,'날짜':date, '리뷰':review })
df

,평점,날짜,리뷰
0,9,2019.12.31 09:48,미드웨이가 재밌으면 추천 백두산이 재밌으면 비추
1,10,2019.12.31 10:41,저 해전이 있었기에 우리나라 광복도 가능 했음
2,10,2019.12.31 09:38,백두산 상영관 대폭줄이고 미드웨이 상영관 대폭늘려라
3,10,2019.12.31 09:14,방금 개봉했는데 1점 준애는 뭐냐 ㅋㅋ 일본놈이냐? 이제 광고돌고 있을 시간이다.
4,10,2019.12.31 19:58,관람객
5,10,2019.12.31 09:29,스케일 겁나커서 보는내내 입벌리고 봣네욬ㅋㅋ재밋습니다
6,10,2019.12.31 09:52,안보면후회함 백두산보다낫다
7,10,2019.12.31 09:01,재밌으면 추천 재미없으면 비추천 눌러주십쇼 형님들
8,10,2019.12.31 16:37,일본의 최대 실수는 미국을 건드린거다 그 대가로 원폭 처맞고 멸망함ㅋㅋㅋ지금 하는 ...
9,8,2019.12.31 10:14,아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 ...


In [20]:
li9 = lis[9]
li9

<li class="last">
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:80.0%"></span></span><em>8</em>
</div>
<div class="score_reple">
<p>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_9">
<span class="_unfold_ment" id="_unfold_ment9">
<a data-src="아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로 20년전에 미드웨이 영화화 반대했고3.평론가 평가가 안좋다?..진주만 보다 20점 높은 44점입니다.(전쟁 영화로써 좋은 점수입니다)4.고증이 안좋다?..스토리 고증은 정말 철저하고 전투기 등등의 디테일은 비슷하지만 완전 정확하진 않음5.중국이 나온다?:나옵니다..하지만 역사적 사실이고 일본은 당시에 미군을 도와준 중국을 보복했죠.(영화는 소니의 반대로 뒤늦게 20년만에 독립영화로 만들어졌는데 1100억의 제작비로 이정도로 강렬한 운명의 5분 미드웨이 해전을 보여준건 놀랍네요) " href="javascript:void(0);" onclick="unfoldPointMent(this);">
																		아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로...
																	</a>
</span>
</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(16564157, 'a

In [21]:
li9.find(class_='score_reple').find('a').string.strip(' \t\r\n')

'아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 장면 하나하나가 거의 다 사실인건 솔직히 놀랍네요)(영화에 대한 팩트들)1.제작비는 500억이 아니라 1100억이고 2.소니가 진짜로...'

In [25]:
len(li.find(class_='score_reple').find_all('span'))

3

In [26]:
len(li9.find(class_='score_reple').find_all('span'))

3

In [36]:
point = []
review = []
date = []
for li in lis:
    score = li.find(class_='star_score').find('em').get_text()
    score = int(score)
    print(score)
    point.append(score)
    count_span = len(li.find(class_='score_reple').find_all('span'))
    if count_span == 2:
        r = li.find(class_='score_reple').find('p').find('span').string.strip(' \t\r\n')
        review.append(r)
    else:
        r = li.find(class_='score_reple').find('a').string.strip(' \t\r\n')
        review.append(r)
    date.append(li.find(class_='score_reple').find_all('em')[1].get_text())

9
10
10
10
10


AttributeError: 'NoneType' object has no attribute 'strip'

In [37]:
li4 = lis[4]
li4

<li>
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:100.0%"></span></span><em>10</em>
</div>
<div class="score_reple">
<p>
<span class="ico_viewer">관람객</span>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_4">
														
															
															
																교과서에는 미드웨이 해전에서 미국이 승리했다고 한 줄로 서술되어 있지만 단순히 한 줄로 설명할 수 없는 파란만장했던 전쟁. 
															
														
														
													</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(16566092, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>꼬깔콘의마술사(topm****)</span>
</a>
</em>
<em>2019.12.31 19:58</em>
</dt>
<dd>
<a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','topm****', 'qLwG0mnvV5pRkeBnIFdhwMdAA0icpeglzYioq2Zq4RQ=', '교과서에는 미드웨이 해전에서 미국이 승리했다고 한 줄로 서술되어 있지만 단순히 한 줄로 설명할 수 없는 파란만장했던 전쟁. ', '16566092', 'point_after', false);return false;"><em>신고</em

In [38]:
len(li4.find(class_='score_reple').find_all('span'))

3

In [39]:
len(li4.find_all(class_='ico_viewer'))

1

In [40]:
len(li9.find_all(class_='ico_viewer'))

0

In [41]:
point = []
review = []
date = []
for li in lis:
    score = li.find(class_='star_score').find('em').get_text()
    score = int(score)
    print(score)
    point.append(score)
    count_span = len(li.find(class_='score_reple').find_all('span'))
    count_icon = len(li.find_all(class_='ico_viewer'))
    if count_span == 2:
        r = li.find(class_='score_reple').find('p').find('span').string.strip(' \t\r\n')
    elif count_icon == 1:
        r = li.find(class_='score_reple').find('p').find_all('span')[1].string.strip(' \t\r\n')
    else:
        r = li.find(class_='score_reple').find('a').string.strip(' \t\r\n')
    review.append(r)
    date.append(li.find(class_='score_reple').find_all('em')[1].get_text())

9
10
10
10
10
10
10
10
10
8


In [42]:
import pandas as pd
df = pd.DataFrame({'평점':point, '일시':date, '감상평':review})
df

,평점,일시,감상평
0,9,2019.12.31 09:48,미드웨이가 재밌으면 추천 백두산이 재밌으면 비추
1,10,2019.12.31 10:41,저 해전이 있었기에 우리나라 광복도 가능 했음
2,10,2019.12.31 09:38,백두산 상영관 대폭줄이고 미드웨이 상영관 대폭늘려라
3,10,2019.12.31 09:14,방금 개봉했는데 1점 준애는 뭐냐 ㅋㅋ 일본놈이냐? 이제 광고돌고 있을 시간이다.
4,10,2019.12.31 19:58,교과서에는 미드웨이 해전에서 미국이 승리했다고 한 줄로 서술되어 있지만 단순히 한 ...
5,10,2019.12.31 09:29,스케일 겁나커서 보는내내 입벌리고 봣네욬ㅋㅋ재밋습니다
6,10,2019.12.31 09:52,안보면후회함 백두산보다낫다
7,10,2019.12.31 09:01,재밌으면 추천 재미없으면 비추천 눌러주십쇼 형님들
8,10,2019.12.31 16:37,일본의 최대 실수는 미국을 건드린거다 그 대가로 원폭 처맞고 멸망함ㅋㅋㅋ지금 하는 ...
9,8,2019.12.31 10:14,아니 에머리히 감독이 스토리에 고증을 철저히 지키다니 안믿긴다..(진짜 실제인물에 ...
